In [1]:
import pandas as pd
import numpy as np

# pip install sentence-transformers (it takes a while to load)
from sentence_transformers import SentenceTransformer, util

2025-03-17 13:26:23.530473: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('rawdata.csv')

# get rid of 'data.' prefix
df.columns = df.columns.str.replace('data.', '')

In [3]:
'''
Documentation: https://huggingface.co/sentence-transformers

TODO: Have a threshold where if the score is too low it is rejected
'''

model = SentenceTransformer("all-MiniLM-L6-v2")

stat_mapping = {
    "points": "points",
    "score": "points",
    "assists": "assists",
    "rebounds": "totalReb",
    "total rebounds": "totalReb",
    "boards": "totalReb",
    "offensive rebounds": "offReb",
    "defensive rebounds": "defReb",
    "steals": "steals",
    "blocks": "blocks",  
    "turnovers": "turnovers",
}

stats = list(stat_mapping)
stat_embeddings = model.encode(stats, convert_to_tensor=True)
names = df['player.name']
name_embeddings = model.encode(names, convert_to_tensor=True)

In [25]:

def get_stat(query):
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    similarities = util.pytorch_cos_sim(query_embedding, stat_embeddings).squeeze(0)
    
    match_index = np.argmax(similarities.numpy())
    score = similarities[match_index].item() 

    #print(score)
    #print(stat_mapping[stats[match_index]])
    return match_index



def get_name(query):
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    similarities = util.pytorch_cos_sim(query_embedding, name_embeddings).squeeze(0)
    
    match_index = np.argmax(similarities.numpy())
    score = similarities[match_index].item()

    #print(score)
    #print(names[match_index])

    return match_index

def output(query):
    stat_index = get_stat(query)
    stat = stats[stat_index]

    name_index = get_name(query)
    name = names[name_index]

    result = df.query(f"`player.name` == '{name}'")[stat_mapping[stat]].iloc[0]

    print(f"{name} had {result} {stat}")


In [26]:
query = "How many rebounds would you say Isaac McKneely had this year"
output(query)

Isaac McKneely had 87 total rebounds
